In [2]:
import pandas as pd
import string
import numpy as np
from sklearn.preprocessing import MinMaxScaler

In [3]:
df = pd.read_csv("data.csv")

In [4]:
print(df)

      Unnamed: 0    file_name  \
0              0  0000f00.fpg   
1              1  0000f01.fpg   
2              2  0000f02.fpg   
3              3  0000f03.fpg   
4              4  0000f04.fpg   
5              5  0000f05.fpg   
6              6  0000f06.fpg   
7              7  0000f07.fpg   
8              8  0000f08.fpg   
9              9  0000f09.fpg   
10            10  0000f10.fpg   
11            11  0000f11.fpg   
12            12  0000f12.fpg   
13            13  0000f13.fpg   
14            14  0000f14.fpg   
15            15  0000f15.fpg   
16            16  0000f16.fpg   
17            17  0000f17.fpg   
18            18  0000f18.fpg   
19            19  0000f19.fpg   
20            20  0000f20.fpg   
21            21  0000f21.fpg   
22            22  0000f22.fpg   
23            23  0000f23.fpg   
24            24  0000f24.fpg   
25            25  0001f00.fpg   
26            26  0001f01.fpg   
27            27  0001f02.fpg   
28            28  0001f03.fpg   
29        

In [5]:
del(df['Unnamed: 0'])
#df.rename(columns={"Unnamed: 0": "t"})

In [6]:
# Removing redundant .fpg extension to avoid confusion
for i in range(len(df['file_name'])):
    df["file_name"][i] = df["file_name"][i].split('.fpg',1)[0]

In [7]:
print(df["file_name"][38])

0001f13


In [8]:
df.columns = ['File Name','X','Y','Pressure','Azumith','Pen Elevation']

In [10]:
colnames = list(df.columns)
filenames = list(df['File Name'])
def clean(t):
  bad = ['.', '\n', '[',']'] 
  t = ''.join(i for i in t if not i in bad) 
  t = t.split(' ')
  t = list(filter(None, t)) 
  t = [float(i) for i in t]
  return(t)
  
df = df.set_index([colnames[0]])
for i in range(1,6):
  df[colnames[i]] = df[colnames[i]].map(clean)

In [11]:
data = {}
auth = {}
scaler = MinMaxScaler()
for timeseries in filenames:
  data[timeseries] = pd.DataFrame()
  auth[timeseries] = 0 if timeseries[4]=='f' else 1 
  for j in range(1,6):
    data[timeseries][colnames[j]] = df[colnames[j]][timeseries]
  offset = data[timeseries].min(axis = 0)[0:2]
  data[timeseries]['X'] -= offset[0]
  data[timeseries]['Y'] -= offset[1]
  data[timeseries][colnames[1:3]] = pd.DataFrame(scaler.fit_transform(data[timeseries][colnames[1:3]]), columns=colnames[1:3])

In [ ]:
print(data["0001f13"])

In [12]:
# Creating separate label dictionary to store labels(1 or 0 for true or false signature respectively) 
# with the format {file_name: label}
label = {}
for key in data:
    if(key.find('f') != -1):
        label[key] = 0
    elif(key.find('v') != -1):
        label[key] = 1
print(label["0001v13"])  #should print 1
print(label["0001f13"])  #should print 0

1
0


In [13]:
print(data["0001f14"])

            X         Y  Pressure  Azumith  Pen Elevation
0    0.052969  0.084225      56.0    167.0           50.0
1    0.028193  0.020053      60.0    165.0           50.0
2    0.020931  0.012032      96.0    164.0           50.0
3    0.018795  0.000000     141.0    163.0           50.0
4    0.018795  0.000000     222.0    163.0           50.0
5    0.023067  0.010695     267.0    161.0           50.0
6    0.027339  0.029412     274.0    161.0           50.0
7    0.039299  0.072193     325.0    161.0           50.0
8    0.055959  0.132353     340.0    160.0           50.0
9    0.075182  0.193850     351.0    160.0           50.0
10   0.093550  0.272727     379.0    160.0           50.0
11   0.111918  0.364973     386.0    159.0           51.0
12   0.128150  0.478610     391.0    159.0           51.0
13   0.138830  0.624332     396.0    159.0           51.0
14   0.147373  0.791444     396.0    158.0           52.0
15   0.154208  0.911765     394.0    158.0           52.0
16   0.158052 

In [14]:
len(data["0001f13"])

277

In [24]:
# Finding minimum number of rows in the dataframe to calclute sampling rate
# #Anomally
# del(data["0091f09"])

for key in data:
    if(len(data[key]) < 200 or len(data[key]) > 600):
        del(data[key])
        #print(data[key])
# print(m)   #m = 1000
# Anomally


RuntimeError: dictionary changed size during iteration

In [23]:
m = 601
for key in data:
    if(len(data[key]) < m):
        m = len(data[key])
print(m)

6


In [16]:
data["0001f13"]['time'] = np.arange(len(data['0001f13']))
print(data["0001f13"])

            X         Y  Pressure  Azumith  Pen Elevation  time
0    0.014974  0.263484      24.0    152.0           57.0     0
1    0.006070  0.180371      29.0    151.0           58.0     1
2    0.000000  0.189213      54.0    151.0           58.0     2
3    0.000000  0.176835     103.0    150.0           58.0     3
4    0.000000  0.169761     142.0    150.0           58.0     4
5    0.005666  0.169761     181.0    150.0           58.0     5
6    0.010117  0.169761     225.0    150.0           58.0     6
7    0.014164  0.177719     264.0    150.0           58.0     7
8    0.025900  0.194518     302.0    150.0           58.0     8
9    0.042898  0.231653     334.0    152.0           59.0     9
10   0.061109  0.281167     399.0    151.0           60.0    10
11   0.079320  0.338638     412.0    151.0           60.0    11
12   0.095103  0.410256     414.0    151.0           60.0    12
13   0.108863  0.503095     448.0    151.0           60.0    13
14   0.124241  0.610080     442.0    152

In [ ]:
# # Upsampling 0001f13 file to 1000 rows
# t = pd.to_timedelta(data["0001f13"].time, unit='S')
# s = data["0001f13"].set_index(t).groupby('X').resample('3S').last().reset_index(drop=True)
# #s.assign(time=s.groupby('X').cumcount())
# print(s)
# print(t)

In [ ]:
# s = pd.Series(index=['X', 'Y', 'Pressure', 'Azumith', 'Pen Elevation', 'time'])
# t = pd.Series([1,2,3,4,5,500], index=['X', 'Y', 'Pressure', 'Azumith', 'Pen Elevation', 'time'])
# print(s)
# print(t)

In [ ]:
# data["0001f13"].append(s, ignore_index=True)
# data["0001f13"].append(t, ignore_index=True)

In [ ]:
# l = 1000-len(data["0001f13"])
# print(l)
# for i in range(l):
#     data['0001f13'].append(s, ignore_index=True)

In [ ]:
# #data['0001f13'].append(s, ignore_index=True)
# data['0001f13'].interpolate()